In [3]:
import tidalapi
import collections
import MySQLdb
import datetime

In [4]:
datetime = datetime.datetime.now()

In [5]:
with open('/home/giorgos/tmp/tidal.txt') as cred:
    credentials = cred.read().splitlines()

In [6]:
USER = credentials[0]
PASS = credentials[1]

In [7]:
session = tidalapi.Session()
session.login(USER, PASS)

True

In [8]:
db = MySQLdb.connect(host='localhost', user='gd', passwd='password', db='tidal')

In [9]:
cur = db.cursor()

In [ ]:
# Necessary configuration to avoid unicode errors from unsupported characters
db.set_character_set('utf8')
cur.execute('SET NAMES utf8;')
cur.execute('SET CHARACTER SET utf8;')
cur.execute('SET character_set_connection=utf8;')

In [ ]:
crtbl = """CREATE TABLE IF NOT EXISTS downloaded_albums
            (id INT AUTO_INCREMENT PRIMARY KEY,
             album_id INT,
             album_name VARCHAR(100),
             album_artist VARCHAR(100),
             dl_date DATE)"""

In [ ]:
cur.execute(crtbl)

In [ ]:
# 1. Select a seed artist ID
#artist = session.get_artist(3116) pearl jam
#artist = session.get_artist(9366) # rem
artist = session.get_artist(64518) # radiohead

In [ ]:
# 2. Find all similar artists
def similar_artists(artist_id):
    sa = session.get_artist_similar(artist_id)
    return [artist.id for artist in sa]

In [ ]:
# 3. Get the top tracks for each artist
def top_tracks(artist_id):
     return session.get_artist_top_tracks(artist_id)

In [ ]:
# 4. Get the top x albums based on the top tracks
skipwords = ['live','greatest hits', 'best of', 'hits']

def top_albums(artist_id, n):
    top_t = top_tracks(artist_id)
    top_a = collections.Counter([track.album.id for track in top_t 
                                 if not any(x in track.album.name.lower() for x in skipwords)
                                 and track.album.id not in dlded])
    if len(top_a)>0:
        return zip(*top_a.most_common(n))[0]
    else:
        return []

In [1]:
def tidal_url(id):
    return 'https://listen.tidal.com/album/{}'.format(id)

In [ ]:
cur.execute("SELECT album_id FROM downloaded_albums")

In [ ]:
# -surely there must be a better way to do this
try:
    dlded = zip(*cur.fetchall())[0]
except IndexError:
    dlded = set()

In [ ]:
#[top_albums(id, 3) for id in similar_artists(artist.id)]
album_ids = set()
for aid in similar_artists(artist.id):
    # put everything in a tuple to skip duplicates
    [album_ids.add(id) for id in top_albums(aid, 5)]

In [ ]:
for aid in album_ids:
    if aid not in dlded:
        album = session.get_album(aid)
        cur.execute("INSERT INTO downloaded_albums\
                (album_id, album_name, album_artist, dl_date) VALUES (%s,%s,%s,%s)",
                (album.id, album.name, album.artist.name, datetime))
        print tidal_url(aid)
db.commit()